In [0]:
load_file = "banktweetsDataList.txt"
#load_file = "RBC_googleplaydata.txt.txt"
train_test_split = 3000

In [97]:
import nltk
nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

##SENTIMENT ANALYSIS

In [98]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

if __name__ == "__main__":

    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)
    print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    sem_classifier = NaiveBayesClassifier.train(train_data)

    print("Accuracy is:", classify.accuracy(sem_classifier, test_data))

    print(sem_classifier.show_most_informative_features(10))

    custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."

    custom_tokens = remove_noise(word_tokenize(custom_tweet))

    print(custom_tweet, sem_classifier.classify(dict([token, True] for token in custom_tokens)))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]
Accuracy is: 0.9966666666666667
Most Informative Features
                      :( = True           Negati : Positi =   2057.5 : 1.0
                      :) = True           Positi : Negati =   1004.7 : 1.0
                follower = True           Positi : Negati =     36.4 : 1.0
                     sad = True           Negati : Positi =     24.1 : 1.0
                 awesome = True           Positi : Negati =     19.0 : 1.0
                     bam = True           Positi : Negati =     17.7 : 1.0
                   enjoy = True           Positi : Negati =     15.9 : 1.0
                    hurt = True           Negati : Positi =     15.0 : 1.0
                  arrive = True           Positi : Negati =     13.9 : 1.0
                     ugh = True           Negati : Positi =     13.6 : 1.0
None
I ordered just once from TerribleCo, 

In [99]:
custom_tweet = "My daughter has been at MEM airport for almost 7 hours trying to fly #unitedAIRLINES to houston. #unitedair what are you going to do???"

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

My daughter has been at MEM airport for almost 7 hours trying to fly #unitedAIRLINES to houston. #unitedair what are you going to do??? Negative


##Technical and Non techincal extraction

In [100]:
from nltk.tokenize import sent_tokenize, word_tokenize

#with open("rbcDataList.txt") as f:
with open(load_file) as f:
  lines = f.readlines()
print(lines[1])

technical_tweets = []
non_technical_tweets = []

banking_tweets = []
insurance_tweets = []
bi_unkown = []

online_tweets = []
offline_tweets = []
oo_unkown = []

tokens_technical_tweets = []
tokens_non_technical_tweets = []

all_terms = techincal_terms + onsite + online + banking_terms + insurance_terms

for line in lines:
  lowercase = line.lower()
  tech = False
  for term in all_terms:
    if term in line:
      technical_tweets.append(line)
      tokens_technical_tweets.append(word_tokenize(line))
      tech = True
  if tech == False:
      non_technical_tweets.append(line)
      tokens_non_technical_tweets.append(word_tokenize(line))

print(tokens_non_technical_tweets)

clean_technical_tweets = []
clean_non_technical_tweets = []

for tokens in tokens_technical_tweets:
  clean_technical_tweets.append(remove_noise(tokens, stop_words))

for tokens in tokens_non_technical_tweets:
  clean_non_technical_tweets.append(remove_noise(tokens, stop_words))

technical_tokens_for_model = get_tweets_for_model(clean_technical_tweets)
non_technical_tokens_for_model = get_tweets_for_model(clean_non_technical_tweets)

technical_dataset = [(tweet_dict, "Technical") for tweet_dict in technical_tokens_for_model]
non_technical_dataset = [(tweet_dict, "Not Technical") for tweet_dict in non_technical_tokens_for_model]

dataset = technical_dataset + non_technical_dataset

random.shuffle(dataset)

train_data = dataset[:train_test_split]
test_data = dataset[train_test_split:]

tech_classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(tech_classifier, test_data))

print(tech_classifier.show_most_informative_features(10))


@somyadixit5 @ScotiabankHelps @scotiabank @BMO @RBC @TD_Canada @cibc #scotiabank is the worst!!!!

[['The', 'rally', 'ends', 'with', 'encouragement', 'for', 'people', 'to', 'attend', 'both', 'the', 'Vigil', 'at', '@', 'DalhousieU', 'for', 'the', 'Canadians', 'killed', 'in', 'Ir\\xe2\\x80\\xa6', 'https', ':', '//t.co/64B3uc8vDe'], ['@', 'briangm11', '@', 'maureen_boland', '@', 'CTVToronto', '@', 'RBC', 'Read', 'the', 'article', 'and', 'find', 'out'], ['re', 'not', 'rich', ',', 'they', 'don'], ['Another', 'great', 'seminar', 'for', 'you', 'in', 'our', 'RBC', 'Seminar', 'Room', '1', '!', 'Learn', 'Franchise', 'Financing', 'at', '1:30', 'PM', '.', 'Presented', 'by', ':', 'Sandro\\xe2\\x80\\xa6', 'https', ':', '//t.co/kwWgKSemsC'], ['@', 'allisoncouto', '@', 'RBC', 'Very', 'common', 'RBC', 'experience.\\nRun', 'away', 'as', 'far', 'as', 'you', 'can', 'from', 'them'], ['@', 'RBC', '@', '_TylerMcIntyre', 'Trash'], ['all', 'a', 'piece', 'of', 'work', '.', 'A', 'woman'], ['Women', 'in', 'Econom

In [101]:
custom_tweet = "Hey @RBC. You may need to update your website for this credit card lol. 2017 is a long time ago. https://t.co/r80qf9FuWr"

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(custom_tweet, tech_classifier.classify(dict([token, True] for token in custom_tokens)))

Hey @RBC. You may need to update your website for this credit card lol. 2017 is a long time ago. https://t.co/r80qf9FuWr Technical


##BANKING VS INSURANCE

In [102]:
banking_tweets = []
insurance_tweets = []
bi_unkown = []

token_banking_tweets = []
token_insurance_tweets = []
token_bi_unkown = []

for tweet in technical_tweets:
  line = tweet.lower()
  match = False
  for term in banking_terms:
    if term in line:
      banking_tweets.append(line)
      token_banking_tweets.append(word_tokenize(line))
      match = True
  for term in insurance_terms:
    if term in line:
      insurance_tweets.append(line)
      token_insurance_tweets.append(word_tokenize(line))
      match = True
  if match == False:
    bi_unkown.append(line)
    token_bi_unkown.append(word_tokenize(line))

clean_banking_tweets = []
clean_insurance_tweets = []
clean_bi_unkown = []

for tokens in token_banking_tweets:
  clean_banking_tweets.append(remove_noise(tokens, stop_words))

for tokens in token_insurance_tweets:
  clean_insurance_tweets.append(remove_noise(tokens, stop_words))

for tokens in token_bi_unkown:
  clean_bi_unkown.append(remove_noise(tokens, stop_words))

token_banking_tweets_model = get_tweets_for_model(clean_banking_tweets)
token_insurance_tweets_model = get_tweets_for_model(clean_insurance_tweets)
token_bi_unkown_model = get_tweets_for_model(clean_bi_unkown)

banking_dataset = [(tweet_dict, "Banking") for tweet_dict in token_banking_tweets_model]
insurance_dataset = [(tweet_dict, "Insurance") for tweet_dict in token_insurance_tweets_model]
bi_unkown_dataset = [(tweet_dict, "Unkown") for tweet_dict in token_bi_unkown_model]

dataset = banking_dataset + insurance_dataset + bi_unkown_dataset

random.shuffle(dataset)

train_data = dataset[:train_test_split]
test_data = dataset[train_test_split:]

bank_classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(bank_classifier, test_data))

print(bank_classifier.show_most_informative_features(10))


Accuracy is: 0.7636837532580365
Most Informative Features
                    file = True           Insura : Bankin =     38.1 : 1.0
               insurance = True           Insura : Bankin =     38.1 : 1.0
                    life = True           Insura : Bankin =     36.0 : 1.0
                    sell = True           Insura : Bankin =     34.5 : 1.0
                     yet = True           Insura : Unkown =     33.3 : 1.0
                   small = True           Insura : Unkown =     27.0 : 1.0
                activate = True           Insura : Bankin =     23.6 : 1.0
                aeroplan = True           Insura : Bankin =     23.6 : 1.0
                    trip = True           Insura : Unkown =     20.6 : 1.0
        we\xe2\x80\x99re = True           Insura : Bankin =     20.0 : 1.0
None


##ONLINE VS ONSITE

In [103]:
online_tweets = []
onsite_tweets = []
oo_unkown = []

token_online_tweets = []
token_onsite_tweets = []
token_oo_unkown = []

for tweet in technical_tweets:
  line = tweet.lower()
  match = False
  for term in online_terms:
    if term in line:
      online_tweets.append(line)
      token_online_tweets.append(word_tokenize(line))
      match = True
  for term in onsite_terms:
    if term in line:
      onsite_tweets.append(line)
      token_onsite_tweets.append(word_tokenize(line))
      match = True
  if match == False:
    oo_unkown.append(line)
    token_oo_unkown.append(word_tokenize(line))

clean_online_tweets = []
clean_onsite_tweets = []
clean_oo_unkown = []

for tokens in token_online_tweets:
  clean_online_tweets.append(remove_noise(tokens, stop_words))

for tokens in token_onsite_tweets:
  clean_onsite_tweets.append(remove_noise(tokens, stop_words))

for tokens in token_oo_unkown:
  clean_oo_unkown.append(remove_noise(tokens, stop_words))

token_online_tweets_model = get_tweets_for_model(clean_online_tweets)
token_onsite_tweets_model = get_tweets_for_model(clean_onsite_tweets)
token_oo_unkown_model = get_tweets_for_model(clean_oo_unkown)

online_dataset = [(tweet_dict, "Online") for tweet_dict in token_online_tweets_model]
onsite_dataset = [(tweet_dict, "Onsite") for tweet_dict in token_onsite_tweets_model]
oo_unkown_dataset = [(tweet_dict, "Unkown") for tweet_dict in token_oo_unkown_model]

dataset = online_dataset + onsite_dataset + oo_unkown_dataset

random.shuffle(dataset)

train_data = dataset[:train_test_split]
test_data = dataset[train_test_split:]

online_classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(online_classifier, test_data))

print(online_classifier.show_most_informative_features(10))


Accuracy is: 0.82015065913371
Most Informative Features
                    cash = True           Onsite : Online =     37.1 : 1.0
          si\xe2\x80\xa6 = True           Online : Unkown =     36.4 : 1.0
                doordash = True           Online : Onsite =     28.8 : 1.0
                 generic = True           Online : Unkown =     19.2 : 1.0
                      '' = True           Online : Onsite =     19.1 : 1.0
            globeandmail = True           Onsite : Unkown =     18.1 : 1.0
                     atm = True           Onsite : Online =     18.0 : 1.0
          sc\xe2\x80\xa6 = True           Online : Unkown =     17.8 : 1.0
                 fintech = True           Online : Unkown =     16.7 : 1.0
                function = True           Online : Unkown =     16.4 : 1.0
None


##SPAM VS NO SPAM

In [0]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from math import log, sqrt
import pandas as pd
import numpy as np
import re
%matplotlib inline

In [105]:
mails = pd.read_csv('spam.csv', encoding = 'latin-1')
mails.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [106]:
mails.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)
mails.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [107]:
mails.rename(columns = {'v1': 'labels', 'v2': 'message'}, inplace = True)
mails.head()

,labels,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [108]:
mails['labels'].value_counts()

ham     4825
spam     747
Name: labels, dtype: int64

In [109]:
mails['label'] = mails['labels'].map({'ham': 0, 'spam': 1})
mails.head()

,labels,message,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [110]:
mails.drop(['labels'], axis = 1, inplace = True)
mails.head()

,message,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [0]:
totalMails = 4825 + 747
trainIndex, testIndex = list(), list()
for i in range(mails.shape[0]):
    if np.random.uniform(0, 1) < 0.75:
        trainIndex += [i]
    else:
        testIndex += [i]
trainData = mails.loc[trainIndex]
testData = mails.loc[testIndex]

In [112]:
trainData.reset_index(inplace = True)
trainData.drop(['index'], axis = 1, inplace = True)
trainData.head()

,message,label
0,Ok lar... Joking wif u oni...,0
1,Free entry in 2 a wkly comp to win FA Cup fina...,1
2,U dun say so early hor... U c already then say...,0
3,"Nah I don't think he goes to usf, he lives aro...",0
4,FreeMsg Hey there darling it's been 3 week's n...,1


In [113]:
testData.reset_index(inplace = True)
testData.drop(['index'], axis = 1, inplace = True)
testData.head()

,message,label
0,"Go until jurong point, crazy.. Available only ...",0
1,URGENT! You have won a 1 week FREE membership ...,1
2,I HAVE A DATE ON SUNDAY WITH WILL!!,0
3,Fine if thatåÕs the way u feel. ThatåÕs the wa...,0
4,Is that seriously how you spell his name?,0


In [114]:
trainData['label'].value_counts()

0    3624
1     555
Name: label, dtype: int64

In [0]:
def process_message(message, lower_case = True, stem = True, stop_words = True, gram = 2):
    if lower_case:
        message = message.lower()
    words = word_tokenize(message)
    words = [w for w in words if len(w) > 2]
    if gram > 1:
        w = []
        for i in range(len(words) - gram + 1):
            w += [' '.join(words[i:i + gram])]
        return w
    if stop_words:
        sw = stopwords.words('english')
        words = [word for word in words if word not in sw]
    if stem:
        stemmer = PorterStemmer()
        words = [stemmer.stem(word) for word in words]   
    return words

In [0]:
class SpamClassifier(object):
    def __init__(self, trainData, method = 'tf-idf'):
        self.mails, self.labels = trainData['message'], trainData['label']
        self.method = method

    def train(self):
        self.calc_TF_and_IDF()
        if self.method == 'tf-idf':
            self.calc_TF_IDF()
        else:
            self.calc_prob()

    def calc_prob(self):
        self.prob_spam = dict()
        self.prob_ham = dict()
        for word in self.tf_spam:
            self.prob_spam[word] = (self.tf_spam[word] + 1) / (self.spam_words + \
                                                                len(list(self.tf_spam.keys())))
        for word in self.tf_ham:
            self.prob_ham[word] = (self.tf_ham[word] + 1) / (self.ham_words + \
                                                                len(list(self.tf_ham.keys())))
        self.prob_spam_mail, self.prob_ham_mail = self.spam_mails / self.total_mails, self.ham_mails / self.total_mails 


    def calc_TF_and_IDF(self):
        noOfMessages = self.mails.shape[0]
        self.spam_mails, self.ham_mails = self.labels.value_counts()[1], self.labels.value_counts()[0]
        self.total_mails = self.spam_mails + self.ham_mails
        self.spam_words = 0
        self.ham_words = 0
        self.tf_spam = dict()
        self.tf_ham = dict()
        self.idf_spam = dict()
        self.idf_ham = dict()
        for i in range(noOfMessages):
            message_processed = process_message(self.mails[i])
            count = list() #To keep track of whether the word has ocured in the message or not.
                           #For IDF
            for word in message_processed:
                if self.labels[i]:
                    self.tf_spam[word] = self.tf_spam.get(word, 0) + 1
                    self.spam_words += 1
                else:
                    self.tf_ham[word] = self.tf_ham.get(word, 0) + 1
                    self.ham_words += 1
                if word not in count:
                    count += [word]
            for word in count:
                if self.labels[i]:
                    self.idf_spam[word] = self.idf_spam.get(word, 0) + 1
                else:
                    self.idf_ham[word] = self.idf_ham.get(word, 0) + 1

    def calc_TF_IDF(self):
        self.prob_spam = dict()
        self.prob_ham = dict()
        self.sum_tf_idf_spam = 0
        self.sum_tf_idf_ham = 0
        for word in self.tf_spam:
            self.prob_spam[word] = (self.tf_spam[word]) * log((self.spam_mails + self.ham_mails) \
                                                          / (self.idf_spam[word] + self.idf_ham.get(word, 0)))
            self.sum_tf_idf_spam += self.prob_spam[word]
        for word in self.tf_spam:
            self.prob_spam[word] = (self.prob_spam[word] + 1) / (self.sum_tf_idf_spam + len(list(self.prob_spam.keys())))
            
        for word in self.tf_ham:
            self.prob_ham[word] = (self.tf_ham[word]) * log((self.spam_mails + self.ham_mails) \
                                                          / (self.idf_spam.get(word, 0) + self.idf_ham[word]))
            self.sum_tf_idf_ham += self.prob_ham[word]
        for word in self.tf_ham:
            self.prob_ham[word] = (self.prob_ham[word] + 1) / (self.sum_tf_idf_ham + len(list(self.prob_ham.keys())))
            
    
        self.prob_spam_mail, self.prob_ham_mail = self.spam_mails / self.total_mails, self.ham_mails / self.total_mails 
                    
    def classify(self, processed_message):
        pSpam, pHam = 0, 0
        for word in processed_message:                
            if word in self.prob_spam:
                pSpam += log(self.prob_spam[word])
            else:
                if self.method == 'tf-idf':
                    pSpam -= log(self.sum_tf_idf_spam + len(list(self.prob_spam.keys())))
                else:
                    pSpam -= log(self.spam_words + len(list(self.prob_spam.keys())))
            if word in self.prob_ham:
                pHam += log(self.prob_ham[word])
            else:
                if self.method == 'tf-idf':
                    pHam -= log(self.sum_tf_idf_ham + len(list(self.prob_ham.keys()))) 
                else:
                    pHam -= log(self.ham_words + len(list(self.prob_ham.keys())))
            pSpam += log(self.prob_spam_mail)
            pHam += log(self.prob_ham_mail)
        return pSpam >= pHam
    
    def predict(self, testData):
        result = dict()
        for (i, message) in enumerate(testData):
            processed_message = process_message(message)
            result[i] = int(self.classify(processed_message))
        return result

In [0]:
def metrics(labels, predictions):
    true_pos, true_neg, false_pos, false_neg = 0, 0, 0, 0
    for i in range(len(labels)):
        true_pos += int(labels[i] == 1 and predictions[i] == 1)
        true_neg += int(labels[i] == 0 and predictions[i] == 0)
        false_pos += int(labels[i] == 0 and predictions[i] == 1)
        false_neg += int(labels[i] == 1 and predictions[i] == 0)
    precision = true_pos / (true_pos + false_pos)
    recall = true_pos / (true_pos + false_neg)
    Fscore = 2 * precision * recall / (precision + recall)
    accuracy = (true_pos + true_neg) / (true_pos + true_neg + false_pos + false_neg)

    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F-score: ", Fscore)
    print("Accuracy: ", accuracy)

In [118]:
sc_tf_idf = SpamClassifier(trainData, 'tf-idf')
sc_tf_idf.train()
preds_tf_idf = sc_tf_idf.predict(testData['message'])
metrics(testData['label'], preds_tf_idf)

Precision:  0.8843537414965986
Recall:  0.6770833333333334
F-score:  0.7669616519174041
Accuracy:  0.9432878679109835


In [119]:

pm = process_message('@RBC @WPGHomeShows @Bryan_Baeumler I\xe2\x80\x99m here in branch trying to pay or US dollar RBC credit card and they won\xe2\x80\x99t eve\xe2\x80\xa6 https://t.co/vE8I15uYQ8')
sc_tf_idf.classify(pm)

False

##KEY WORDS

In [0]:
onsite_terms = [
  "branch",
  "reception",
  "cash",
  "atm",
  "person",
  "open",
  "close",
  "appointment",
  "sat",
  "spoke",
  "change"

]

online_terms = [
  "web",
  "click",
  "app",
  "buffer",
  "crash",
  "install",
  "download",
  "down",
  "etransfer",
  "e-transfer",
  "type",
  "password",
  "pass",
  "display",
  "error",
  "message",
  "article",
  "currently"
]

In [0]:
banking_terms = [
  "deposit",
  "withdraw",
  "balance",
  "account",
  "deny",
  "exchange",
  "invest",
  "mutual",
  "fund",
  "tfsa",
  "stock",
  "draft",
  "expense",
  "asset",
  "cash",
  "earnings",
  "bonds",
  "401k",
  "risk",
  "credit",
  "saving",
  "debit",
  "interest",
  "value",
  "market",
  "advisor",
  "dividend",
]

insurance_terms = [
  "coverage",
  "insure",
  "insurance",
  "premium",
  "benefit",
  "deductible",
  "policy",
  "co-pay",
  "copay",
  "slip",
  "fall",
  "life",
  "disability",
  "broker",
  "liability",
  "property",
  "accident",
  "death",
  "plan",
  "claim",
  "dental",
  "medical",
  "drug",
  "annuity",
  "manulife",
  "sunlife",
  "greatwest",
  "desjardins"

]

In [0]:
techincal_terms = [
  "help",
  "product",
  "service",
  "shopping",
  "banking",
  "issue",
  "problem",
  "crash",
  "loan",
  "bank",
  "debt",
  "investment",
  "error",
  "insert",
  "edit",
  "update",
  "delete",
  "create",
  "post",
  "solution",
  "lag",
  "bug",
  "delay",
  "cancel",
  "payment",
  "number",
  "reference",
  "card",
  "receive",
  "data",
  "web",
  "stack",
  "host",
  "manage",
  "analyze",
  "analysis",
  "colaboration",
  "execution",
  "save",
  "application",
  "customize",
  "run",
  "sample",
  "list",
  "test",
  "tools",
  "upload",
  "token",
  "resource",
  "train",
  "model",
  "drive",
  "accuracy",
  "feature",
  "randomize",
  "shuffle",
  "locate",
  "view",
  "file",
  "refresh",
  "express",
  "browse",
  "extensions",
  "library",
  "parser",
  "split",
  "integrate",
  "binary",
  "process",
  "router",
  "port",
  "setup",
  "digital",
  "computer",
  "abstraction",
  "analog",
  "autocomplete",
  "bluetooth",
  "buffer",
  "cache",
  "clean",
  "column",
  "mining",
  "default",
  "defragment",
  "encrytion",
  "field",
  "flowchart",
  "frequencey",
  "reimburse",
  "present",
  "fraud",
  "deposit",
  "withdrawl",
  "balance",
  "account",
  "claim",
  "deny",

]